<a href="https://colab.research.google.com/github/indl1670/KST_Project/blob/maskrcnn/polygonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/kst_project/side/maskrcnn

/content/drive/MyDrive/kst_project/side/maskrcnn


In [ ]:
import os
import zipfile
import shutil
import json

# json 파일 경로 지정
path = input("Input path: ")

# json 파일 경로 리스트에 저장
file_list = os.listdir(path)
file_list_json = [file for file in file_list if file.endswith(".json")]

trn_img = []

# json 파일 전체 탐색
for i in range(len(file_list_json)):
    img_name = []

    # json 파일 로드
    file_path =  path + "/" + file_list_json[i]
    with open(file_path, 'r', encoding='UTF-8') as f:
        data = json.load(f)
        
        
        # 라벨링이 있는 이미지 파일 명 저장
        for e in data['images']:
            for f in data['annotations']:
                if e['id'] == f['image_id']:
                    if f['segmentation'] != []:
                        img_name.append(e['file_name'])
        img_name = set(img_name)
        
    trn_img.extend(img_name)

# 해당 이미지 분리: 총 16031     
print(trn_img)

In [ ]:
import os
import shutil

from sklearn.model_selection import train_test_split


def main():
    json_dir = input('json폴더의 주소: ')

    json_list = os.listdir(json_dir)

    if not os.path.exists(json_dir + '//train'):
        os.makedirs(json_dir + '//train')
    if not os.path.exists(json_dir + '//val'):
        os.makedirs(json_dir + '//val')
    if not os.path.exists(json_dir + '//test'):
        os.makedirs(json_dir + '//test')

    # train:valid:test = 7:2:1
    train_set, test_set = train_test_split(json_list, test_size=0.3, random_state=123)
    valid_set, test_set = train_test_split(test_set, test_size=0.4, random_state=123)

    # json 파일을 이동
    for tr in train_set:
        shutil.copy(os.path.join(json_dir, tr), json_dir + '//train//' + tr)
    for va in valid_set:
        shutil.copy(os.path.join(json_dir, va), json_dir + '//val//' + va)
    for te in test_set:
        shutil.copy(os.path.join(json_dir, te), json_dir + '//test//' + te)


if __name__ == "__main__":
    main()


json폴더의 주소: /content/drive/Shareddrives/KST_Project/JSON


In [ ]:
!pip install pyodi

In [ ]:
import os
import json
import shutil


def coco_merge(json_dir, dest, cls):
    json_list = os.listdir(json_dir)
    file_list_json = [file for file in json_list if file.endswith(".json")]

    theone = os.path.join(json_dir, file_list_json[0])
    thetwo = os.path.join(json_dir, file_list_json[1])
    cls = cls + '.json'
    target = os.path.join(dest, 'annotations', cls)

    os.system('pyodi coco merge %s %s %s' % (theone, thetwo, target))
    els = file_list_json[2:]

    for e in els:
        sac = os.path.join(json_dir, e)
        os.system('pyodi coco merge %s %s %s' % (target, sac, target))


def image_split(dest, data_pick):
    img_dir = input('image 파일들의 경로:')

    ann_dir = os.path.join(dest, 'annotations')
    img_dest = os.path.join(dest, 'images')

    with open(os.path.join(ann_dir, 'train.json'), 'r', encoding='utf8') as ann:
        data = json.load(ann)
        img_list = data['images']
        for img in img_list:
            fn = img['file_name']
            if fn in data_pick:
              if os.path.isfile(os.path.join(img_dir, fn)):
                shutil.copy(os.path.join(img_dir, fn), os.path.join(img_dest, 'train'))
        print('train finish')

    with open(os.path.join(ann_dir, 'val.json'), 'r', encoding='utf8') as ann:
        data = json.load(ann)
        img_list = data['images']
        for img in img_list:
            fn = img['file_name']
            if fn in data_pick:
              if os.path.isfile(os.path.join(img_dir, fn)):
                shutil.copy(os.path.join(img_dir, fn), os.path.join(img_dest, 'val'))
        print('val finish')

    with open(os.path.join(ann_dir, 'test.json'), 'r', encoding='utf8') as ann:
        data = json.load(ann)
        img_list = data['images']
        for img in img_list:
            fn = img['file_name']
            if fn in data_pick:
              if os.path.isfile(os.path.join(img_dir, fn)):
                shutil.copy(os.path.join(img_dir, fn), os.path.join(img_dest, 'test'))
        print('test finish')


def main():
    print('--coco data merge--')
    train_json = input('train json 파일들의 경로: ')
    val_json = input('valid json 파일들의 경로: ')
    test_json = input('test json 파일들의 경로: ')

    dest = input('split한 데이터셋을 저장할 경로: ')

    # 폴더 생성
    if not os.path.exists(dest + '//images'):
        os.makedirs(dest + '//images')
        os.makedirs(dest + '//images//train')
        os.makedirs(dest + '//images//val')
        os.makedirs(dest + '//images//test')
    if not os.path.exists(dest + '//annotations'):
        os.makedirs(dest + '//annotations')

    # # coco를 train/val/test 별로 합치기
    coco_merge(train_json, dest, 'train')
    coco_merge(val_json, dest, 'val')
    coco_merge(test_json, dest, 'test')

    # 원하는 이미지 리스트 넣기 = 위 1.2에서 나온 trn_img를 사용
    data_pick = trn_img

    print('--image split--')
    image_split(dest, data_pick)



if __name__ == "__main__":
    main()


--coco data merge--
train json 파일들의 경로: /content/drive/Shareddrives/KST_Project/JSON/train
valid json 파일들의 경로: /content/drive/Shareddrives/KST_Project/JSON/val
test json 파일들의 경로: /content/drive/Shareddrives/KST_Project/JSON/test
split한 데이터셋을 저장할 경로: /content/drive/Shareddrives/KST_Project/polygon
--image split--
train finish
val finish
test finish


In [ ]:
import json
import os

dir = '/content/drive/Shareddrives/KST_Project/polygon/annotations/test.json'

f = open(dir, 'r', encoding = 'utf8')
data = json.load(f)
f.close()
with open(dir, 'w', encoding='utf8') as w:
  for l in data['annotations']:
    if l['category_id'] == 1:
      l['category_id'] = 6
    elif l['category_id'] == 2:
      l['category_id'] = 5
    elif l['category_id'] == 3:
      l['category_id'] = 4
    elif l['category_id'] == 4:
      l['category_id'] = 3
    elif l['category_id'] == 5:
      l['category_id'] = 2
    elif l['category_id'] == 6:
      l['category_id'] = 1
  json.dump(data, w, indent='\t')

In [ ]:
import os
import json

json_dir = '/content/drive/Shareddrives/KST_Project/polygon/annotations'
img_dir = '/content/drive/Shareddrives/KST_Project/polygon/images'
cls = ['train', 'val', 'test']

for c in cls:
  cnt = 0
  img_name_list = os.listdir(os.path.join(img_dir, c))
  with open(os.path.join(json_dir, c+'.json'), 'r', encoding='utf8') as js:
    data = json.load(js)
    img_list = data['images']
    for img in img_list:
      if img['file_name'] in img_name_list:
        cnt+=1
    print(c, 'image: ', len(img_name_list))
    print(c, 'image from json: ', len(img_list))
    print(c, 'match image: ', cnt)


In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [9]:
cd /content/drive/MyDrive/kst_project/oneCycle/mmdetection

/content/drive/MyDrive/kst_project/oneCycle/mmdetection


In [3]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.10.0+cu111
cuda version: 11.1
cudnn version:8005


In [ ]:
!pip install openmim
!mim install mmdet==2.19.0
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

In [ ]:
!git clone --branch v2.19.0 https://github.com/open-mmlab/mmdetection.git

In [ ]:
!pip install -r requirements/build.txt
!python setup.py develop

In [ ]:
# for instaboost
!pip install instaboostfast
# for panoptic segmentation
!pip install git+https://github.com/cocodataset/panopticapi.git
# for LVIS dataset
!pip install git+https://github.com/lvis-dataset/lvis-api.git
# for albumentations
!pip install albumentations>=0.3.2 --no-binary imgaug,albumentations

In [19]:
!python tools/train.py configs/_base_/default_runtime.py

2021-12-19 14:32:10,453 - mmdet - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
CUDA available: True
GPU 0: Tesla K80
CUDA_HOME: /usr/local/cuda
NVCC: Build cuda_11.1.TC455_06.29190527_0
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.10.0+cu111
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=